#### Setup


In [1]:
import torch
from data_handlers import Load_ImageNet100
from overcomplete.models import DinoV2, ViT, ResNet
from torch.utils.data import DataLoader, TensorDataset
from overcomplete.sae import TopKSAE, train_sae
from overcomplete.visualization import (overlay_top_heatmaps, evidence_top_images, zoom_top_images, contour_top_image)
import os
import matplotlib.pyplot as plt
from einops import rearrange
from universal_trainer import train_usae
from activation_generator import Load_activation_dataloader
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Train a USAE

Define Models and Generate Their Activations

In [2]:
models = {
    "DinoV2": DinoV2(device="cuda"),
    "ViT": ViT(device="cuda")
}

activation_dir = "activations/ImageNet100_Dino_ViT"
image_loader = Load_ImageNet100(transform=None, batch_size=256, shuffle=True)

activations_dataloader = Load_activation_dataloader(
    models=models,
    image_dataloader=image_loader, 
    save_dir=activation_dir, 
    generate=False,  
    rearrange_string='n t d -> (n t) d'
    )

Using cache found in C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SAE Visualization

In [3]:
concepts = 1024
epochs = 20
lr=5e-4
sample = next(iter(activations_dataloader))

In [4]:
sample['activations_DinoV2'].shape

torch.Size([1, 65536, 384])

In [5]:
SAEs = {}
optimizers = {}

for key, model in models.items():
  SAEs[key] = TopKSAE(sample[f"activations_{key}"].shape[-1], nb_concepts=concepts, top_k=3, device='cuda')
  optimizers[key] = torch.optim.Adam(SAEs[key].parameters(), lr=lr)

criterion = nn.MSELoss()

train_usae(names=list(models.keys()),
           models=SAEs,
           dataloader=activations_dataloader,
           criterion=criterion,
           nb_epochs=epochs,
           optimizers=optimizers,
           device='cuda')

Epoch 1/20:   0%|          | 0/127 [00:00<?, ?it/s]c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([50176, 768])) that is different to the input size (torch.Size([65536, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/20:   0%|          | 0/127 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (65536) must match the size of tensor b (50176) at non-singleton dimension 0

In [17]:
# now the funny part, we have access to 4 functions that allow us to inspect the concepts,
# let use them to understand a bit more the top 3 concepts !

sae = sae.eval()

with torch.no_grad():
  pre_codes, codes = sae.encode(Activations)


print("Raw Code Shape: ", codes.shape)
codes = rearrange(codes, '(n w h) d -> n w h d', w=14, h=14)
print("Rearranged Code Shape: ", codes.shape)


Raw Code Shape:  torch.Size([392000, 384])
Rearranged Code Shape:  torch.Size([2000, 14, 14, 384])


In [18]:
# Compute per-concept L1 norm across all spatial positions and batch
codes_flat = codes.abs().sum(dim=(1, 2))        # (32, D)
concept_strength = codes_flat.sum(dim=0)        # (D,)

topk = int((1 - logs['dead_features'][-1]) * concepts)
top_concepts = torch.argsort(concept_strength, descending=True)[:topk].to(device)

print(f"Top-{topk} Concepts by L1 Norm")

# first, lets use the simple overlay to get a broad sense of what's going on
for concept_id in top_concepts:
  print('Concept', concept_id.item())
  overlay_top_heatmaps(image, codes, concept_id=concept_id.item())
  os.makedirs(save_dir, exist_ok=True)
  filename = f"concept_{concept_id.item()}.png"
  filepath = os.path.join(save_dir, filename)
  plt.savefig(filepath, bbox_inches='tight', dpi=300)
  plt.close()  

Top-108 Concepts by L1 Norm
Concept 234
Concept 150
Concept 59
Concept 110
Concept 127
Concept 365
Concept 13
Concept 78
Concept 254
Concept 8
Concept 354
Concept 275
Concept 45
Concept 151
Concept 293
Concept 158
Concept 20
Concept 195
Concept 170
Concept 89
Concept 72
Concept 217
Concept 277
Concept 373
Concept 209
Concept 192
Concept 169
Concept 79
Concept 153
Concept 270
Concept 219
Concept 320
Concept 65
Concept 19
Concept 290
Concept 274
Concept 172
Concept 5
Concept 96
Concept 210
Concept 244
Concept 358
Concept 379
Concept 301
Concept 36
Concept 61
Concept 164
Concept 50
Concept 104
Concept 376
Concept 356
Concept 179
Concept 364
Concept 327
Concept 68
Concept 264
Concept 93
Concept 49
Concept 207
Concept 242
Concept 12
Concept 310
Concept 269
Concept 4
Concept 148
Concept 218
Concept 157
Concept 258
Concept 268
Concept 241
Concept 40
Concept 294
Concept 198
Concept 44
Concept 307
Concept 149
Concept 42
Concept 58
Concept 90
Concept 299
Concept 176
Concept 288
Concept 206
Conce